# Plot Individual Traces
**Description:** This notebook visualizes the signals for a single embryo.  

* Dff trace
* Partial dff trace
* Spectrogram
* Hipass filter
* Lowpass filter

## Import Libraries

In [ ]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import matplotlib as mpl

from pasna_analysis import Experiment, myplots

## Load Data

Specify the location of the data by editing `folder` and `experiment_name`. Then specify the embryo.

In [ ]:
folder = '25C' # HERE
experiment_name = '20240919_25C' # HERE

exp_path = Path.cwd().parent.joinpath('data', folder, experiment_name)
exp = Experiment(exp_path)

embryos = list(exp.embryos.values())
emb = embryos[18] # HERE
trace = emb.trace

print(f'\n\nLoading {folder} {experiment_name} {emb.name}')

## Configure Plot Style

Customize the Matplotlib plots with rcParams. For a full list of parameters see [matplotlib.rcParams](https://matplotlib.org/stable/api/matplotlib_configuration_api.html#default-values-and-styling)

In [ ]:
myplots.set_darkbg(False)

trace_rc = {
    'figure.figsize': (35, 6),
    'figure.frameon': False,

    'lines.linewidth': 3.5,

    'font.family': 'Arial',
    'font.size': 40,

    'xtick.major.size': 10, # length
    'xtick.major.width': 2,
    'ytick.major.size': 10, # length
    'ytick.major.width': 2,

    'axes.spines.top': False,
    'axes.spines.right': False,
    'axes.labelpad': 10,
    'axes.linewidth': 2.5
}

partial_trace_rc = {
    'figure.figsize': (20, 8),
    'figure.frameon': False,

    'lines.linewidth': 4,

    'font.family': 'Arial',
    'font.size': 40,

    'xtick.major.size': 10, # length
    'xtick.major.width': 2,
    'ytick.major.size': 10, # length
    'ytick.major.width': 2,

    'axes.spines.top': False,
    'axes.spines.right': False,
    'axes.labelpad': 10,
    'axes.linewidth': 2.5
}

spec_rc = {
    'figure.figsize': (35, 8),
    'figure.frameon': False,

    'font.family': 'Arial',
    'font.size': 40,

    'axes.spines.top': False,
    'axes.spines.right': False,

    'xtick.major.size': 10, # length
    'xtick.major.width': 2,
    'ytick.major.size': 10, # length
    'ytick.major.width': 2,
    'axes.labelpad': 10,
    'axes.linewidth': 2.5
}

mymap = mpl.colormaps['turbo']
mymap.set_over('black')

## Plot DFF

Preprocesses the dff to a standardized length, then plots it over real time.

**preprocess_dff**

optional parameters:
* duration: target number of frames to include
* onset_pad: number of frames to include before onset


**plot_trace** 

optional parameters:
* color: line color
* xmin: x-axis minimum in minutes
* xmax: x-axis maximum in minutes
* xinterval: num of minutes between ticks
* ymin: y-axis minimum in minutes
* ymax: y-axis maximum in minutes
* yinterval: num of minutes between ticks

note: if you dont specify a color, one will be chosen for you

In [ ]:
time, dff = trace.preprocess_dff()
myplots.plot_trace(time, dff, trace_rc, color="#0273b2ff", xmax=300, ymax=1, yinterval=0.5)

## Plot Partial DFF

Preprocesses the dff to a standardized length, then plots it over real time.

**preprocess_dff** 

optional parameters:
* duration: target number of frames to include
* onset_pad: number of frames to include before onset


**plot_trace** 

optional parameters:
* color: line color
* xmin: x-axis minimum in minutes
* xmax: x-axis maximum in minutes
* xinterval: num of minutes between ticks
* ymin: y-axis minimum in minutes
* ymax: y-axis maximum in minutes
* yinterval: num of minutes between ticks

note: if you dont specify a color, one will be chosen for you

In [ ]:
time, dff = trace.preprocess_dff()
myplots.plot_trace(time, dff, partial_trace_rc, color="#0273b2ff", xmin=165, xmax=195, xinterval=5, ymax=1)

## Plot Spectrogram

Preprocesses the dff to a standardized length, then plots the magnitude of its STFT over time. The x-axis displays time (columns), y-axis displays frequency (rows), and color displays intensity. 

**preprocess_dff** 

optional parameters:
* duration: target number of frames to include
* onset_pad: number of frames to include before onset


**calculate_STFT**

optional parameters:
* fs: sampling rate
* fft_size: num frames in each segment
* noverlap: num frames to overlap between segements

**plot_spec** 

optional parameters:
* display_colorbar: True to display, False to hide
* xmin: x-axis minimum in minutes
* xmax: x-axis maximum in minutes
* ymin: y-axis minimum in minutes
* ymax: y-axis maximum in minutes

In [ ]:
time, dff = trace.preprocess_dff()
f, t, Zxx = trace.calculate_STFT(dff)
myplots.plot_spec(f, t, Zxx, mymap, spec_rc, display_colorbar=False)

## Apply Highpass Filter

**apply_hipass_filter**

optional parameters:

* fs: sampling rate
* fft_size: num frames in each segment
* numtaps: num of coefficients in the filter

In [ ]:
cutoff = 0.008 # frequency in Hz

f, t, Zxx = trace.calculate_STFT(dff)
hipass_time, hipass_dff = trace.apply_hipass_filter(time, dff, cutoff)

myplots.plot_trace(hipass_time, hipass_dff, trace_rc, color="cyan", xmax=300, ymin=-0.5, ymax=0.5, yinterval=0.5)

new_f, new_t, new_Zxx = trace.calculate_STFT(hipass_dff)
myplots.plot_spec(new_f, new_t, new_Zxx, mymap, spec_rc, display_colorbar=False, xmax=300)

## Apply Lowpass Filter

**apply_lopass_filter**

optional parameters:

* fs: sampling rate
* fft_size: num frames in each segment
* numtaps: num of coefficients in the filter

In [ ]:
cutoff = 0.008 # frequency in Hz

f, t, Zxx = trace.calculate_STFT(dff)
lopass_time, lopass_dff = trace.apply_lopass_filter(time, dff, cutoff)

myplots.plot_trace(lopass_time, lopass_dff, trace_rc, color="hotpink", xmax=300, ymin=-0.5, ymax=0.5, yinterval=0.5)

new_f, new_t, new_Zxx = trace.calculate_STFT(lopass_dff)
myplots.plot_spec(new_f, new_t, new_Zxx, mymap, spec_rc, display_colorbar=False,  xmax=300)